In [ ]:
# import dependencies
from pathlib import Path
import cv2
import numpy as np

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
def get_YOLOdata(src_path, set='train', n_samples=None):
  '''Loads images and labels from folder containing subfolders of 'images' and 'labels' (YOLO format).
  Args:
  src_path (Path): Path to source folder.
  set (str): Set to be loaded. Either 'train' or 'val'.
  n_samples (int): Number of image samples to load. If None, load all images and their associated labels.
  Return:
  images: List of images, type ndarray.
  labels: List of labels, type ndarray.
  '''
  images, labels = [], []

  # fetch images and labels files
  img_dir = src_path / set / 'images'
  label_dir = src_path / set / 'labels'
  img_files = list(img_dir.glob('*.jpg'))
  label_files = list(label_dir.glob('*.txt'))

  # each image should be matched up with a label
  assert len(img_files) == len(label_files)

  # take n random samples from dataset
  if n_samples:
    sample_idx = np.random.choice(len(img_files), n_samples, replace=False)
    img_files = [img_files[i] for i in sample_idx]
    label_files = [label_files[i] for i in sample_idx]

  for img_file in img_files:
    img = cv2.imread(str(img_file)) # cv2 loads image in BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert to RGB
    images.append(img)

  # YOLO labels as <class_id> <x_center> <y_center> <box_width> <box_height>
  # the values are normalized between 0 and 1, propotional to image size
  for label_file in label_files:
    with open(str(label_file), 'r') as f:
      # each line is a bounding box label
      lines = f.readlines()
      label = [line.strip().split() for line in lines]
      label = np.array(label, dtype=np.float32)
      labels.append(label)

  return images, labels

In [ ]:
# create custom pytorch dataset
class YOLOdataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = torch.from_numpy(self.labels[idx]).float()

        if self.transform:
            img = self.transform(img)
        else:
            img = cv2.resize(img, (224,224))
            img = transforms.ToTensor()(img)
            img = transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)(img)

        return img, label

    # needed since pytorch cannot 'auto-batch' images with varying number of bounding boxes, labels have
    # different tensor sizes -> so instead of torch.stack that pytorch would do automatically, we leave labels as a list of tensors
    def custom_collate_fn(self, batch):
        images, labels = zip(*batch) # unpack batch
        images = torch.stack(images)
        return images, labels


# DataLoaders
def get_loader(src_path, set='train', n_samples=None, batch_size=4, shuffle=True):
    images, labels = get_YOLOdata(src_path, set=set, n_samples=n_samples) # get data from src

    dataset = YOLOdataset(images, labels) # make pytorch dataset

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=dataset.custom_collate_fn) # make dataloader (with pytorch dataset)
    return loader

In [ ]:
src_path = Path(r"C:\Users\emman\OneDrive\Documents\MI Projects\Hot Dog Detection YOLO")